In [1]:
import sys 
sys.path.append('../../')

In [4]:
import Helpers.helpers as helpers

plain_df = helpers.load_dataset('../export/plain.csv');
plain_df.head()

,X,Y
0,0.914339,4.550146
1,-1.930380,2.702150
2,1.872718,4.180692
3,-0.555524,4.695958
4,-1.034436,8.259404


In [19]:
from diffprivlib.mechanisms import laplace

lp = laplace.Laplace(epsilon=0.5, sensitivity=0.5)
lp.randomise(2)

2.511210169229778

In [24]:
import numpy as np

# [https://github.com/forestneo/sunPytools/blob/master/dplib/ldp_mechanisms/piecewise_mechanism.py](https://github.com/forestneo/sunPytools/blob/master/dplib/ldp_mechanisms/piecewise_mechanism.py)
# Collecting and Analyzing Multidimensional Data with Local Differential Privacy
def randomize_piece_wise(epsilon, value):
    z = np.e ** (epsilon / 2)
    C = (z + 1) / (z - 1)
    P1 = (value + 1) / (2 + 2 * z)
    P2 = z / (z + 1)
    P3 = (1 - value) / (2 + 2 * z)

    g1 = (C + 1) * value / 2 - (C - 1) / 2
    g2 = (C + 1) * value / 2 + (C - 1) / 2

    rnd = np.random.random()
    if rnd < P1:
        result = -C + np.random.random() * (g1 - (-C))
    elif rnd < P1 + P2:
        result = (g2 - g1) * np.random.random() + g1
    else:
        result = (C - g2) * np.random.random() + g2
    return result


In [62]:
pw =randomize_piece_wise(3, 1)
pw

1.2026308094512266